In [1]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
"""Initializing Spark Session"""
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
"""Read API That Shows University Information"""
r = requests.request("GET",'http://universities.hipolabs.com/search?country=United+States')
"""Print JSON Fetched file as python dictionary"""
data_j = r.json()
"""Place a list of JSON Objects into a spark dataframe"""
df = sc.parallelize(data_j).map(lambda x: json.dumps(x))
df = spark.read.json(df)
"""The File had a multitude of Null values in the state column which is what I wanted to work with"""
df = df.na.drop(subset=['state-province'])


In [2]:
"""Count the number of universities that appeard in the list per state"""
state_sum = df.groupBy("state-province").count()
state_sum.show(50)

+--------------+-----+
|state-province|count|
+--------------+-----+
|          Utah|    1|
|         Texas|    4|
|  Pennsylvania|   31|
|            NV|    1|
|            VA|    1|
|          Iowa|    1|
|            TX|    2|
|       Indiana|    1|
|          Ohio|    2|
|            IN|    1|
|      New York|    1|
|      Colorado|    1|
|            CA|    2|
|      Illinois|    1|
|North Carolina|    2|
| New Hampshire|    1|
|South Carolina|    1|
|    Washington|    2|
|      Missouri|    1|
|      Virginia|    1|
|    California|    5|
|            NY|    9|
|            MI|    1|
|            ND|    1|
|      Michigan|    1|
|  New York, NY|    4|
|            GA|    2|
|       Florida|    1|
|         Maine|    1|
+--------------+-----+



In [5]:
"""University Name Contains the name of the state"""
state_univers = df.filter(col("name").contains(col("state-province")))
state_univers.show(state_univers.count(),truncate=False)

+--------------+-------------+------------------------------------------+-------------------------------------------------+--------------+--------------------------------+
|alpha_two_code|country      |domains                                   |name                                             |state-province|web_pages                       |
+--------------+-------------+------------------------------------------+-------------------------------------------------+--------------+--------------------------------+
|US            |United States|[utrgv.edu]                               |The University of Texas at Rio Grande Valley     |Texas         |[https://www.utrgv.edu/]        |
|US            |United States|[uvu.edu]                                 |Utah Valley University                           |Utah          |[https://uvu.edu/]              |
|US            |United States|[bloomu.edu]                              |Bloomsburg University of Pennsylvania            |Pennsylvania  |[h